In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from llama_cpp import Llama
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding, LlamaDraftModel
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.chat_models import init_chat_model
from openai import OpenAI


In [3]:
OPENAI_API_KEY = "sk-your-key"
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# llm = init_chat_model("ft:gpt-4o-mini-2024-07-18:tu-graz-hereditary:gutbrain-ie-finetune:B5qr9cGV", model_provider="openai")
llm = init_chat_model("gpt-4o-mini-2024-07-18", model_provider="openai")

In [4]:
model = Llama.from_pretrained(
    "NousResearch/Hermes-3-Llama-3.2-3B-GGUF",
    filename="*.Q8_0.gguf",
    n_gpu_layers=-1,
    n_ctx=8096,
    temperature=0.1,
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4090, compute capability 8.9, VMM: yes
llama_load_model_from_file: using device CUDA0 (NVIDIA GeForce RTX 4090) - 12583 MiB free
llama_model_loader: loaded meta data with 38 key-value pairs and 255 tensors from /home/bkantz/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.2-3B-GGUF/snapshots/3cd927095d8cbab12c743f932aa63b6f7bbfa141/./Hermes-3-Llama-3.2-3B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hermes 3 Llama 3.2 3b Base Fft Chatml...
llama_mod

In [5]:
model_path = "quants/llama-3-2-1B-instruct-lora.gguf"
# model = Llama(
#     model_path,
#     n_gpu_layers=-1,
#     n_ctx=4096,
#     temperature=0.1,
#     # draft_model=LlamaPromptLookupDecoding(num_pred_tokens=10),
# )

In [6]:
from constrerl.annotator import (
    Annotator,
    Article,
    load_train,
    load_test,
    convert_to_enum_model,
    convert_to_string_model,
    article_to_enum_model,
    ExtendedEnumERLModel,
    ExtendedStringERLModel,
)
from constrerl.erl_schema import convert_to_output
from constrerl.ner import NERCleanr

In [7]:
data_path = "data/articles/articles_test.json"
out_path = "data/results/dev_out.json"
annotator = Annotator(
    model=model,
    gen_tokens=2048,
    add_rag=True,
    reorder=False,
    top_k=5,
    add_entity_labels=True,
)
ner = NERCleanr(
    model=model,
    gen_tokens=1024,
    add_rag=True,
    top_k=5,
    add_few_shot=False,
    add_entity_labels=True,
)
# annotator = Annotator(model=model, gen_tokens=2048)
eval_set = load_test(data_path)
few_shot_samples = 0
# annotator.add_prompt_examples([a for a in eval_set.values()][0:few_shot_samples])

In [8]:
with open("grammar.gbnf", "w") as f:
    f.write(annotator.erl_grammar)
print(annotator.erl_grammar)

root ::= (" "| "\n") grammar-models
grammar-models ::= relations
relations ::= "{"  ws "\"relations\"" ": " relations-relations  ws "}"
relation ::= "{"  ws "\"link_type\"" ": " relation-link-type ","  ws "\"subject_text_span\"" ": " string ","  ws "\"subject_location\"" ": " relation-subject-location ","  ws "\"object_text_span\"" ": " string ","  ws "\"object_location\"" ": " relation-object-location  ws "}"
relation-link-type ::= "\"anatomical location | located in | human\"" | "\"anatomical location | located in | animal\"" | "\"bacteria | interact | bacteria\"" | "\"bacteria | interact | chemical\"" | "\"bacteria | interact | drug\"" | "\"bacteria | influence | DDF\"" | "\"bacteria | change expression | gene\"" | "\"bacteria | located in | human\"" | "\"bacteria | located in | animal\"" | "\"bacteria | part of | microbiome\"" | "\"chemical | located in | anatomical location\"" | "\"chemical | located in | human\"" | "\"chemical | located in | animal\"" | "\"chemical | interact | c

In [9]:
annotator.example_messages

[{'role': 'system',
  'content': 'You are annotating a medical scientific title and abstract. You return all relation between entities within the title and abstract as JSON. The returned data include the relation type and text and should cover the most relevant relations occurring in the text.  The possible entities are:\nAnatomical Location: Named locations of or within the body. \nAnimal: A non-human living organism that has membranous cell walls, requires oxygen and organic foods, and is capable of voluntary movement, as distinguished from a plant or mineral.\nBiomedical Technique: Research concerned with the application of biological and physiological principles to clinical medicine.\nBacteria: One of the three domains of life (the others being Eukarya and ARCHAEA), also called Eubacteria. They are unicellular prokaryotic microorganisms which generally possess rigid cell walls, multiply by cell division, and exhibit three principal forms: round or coccal, rodlike or bacillary, and 

In [10]:
test_samples = 10

In [11]:
# annotations = annotator.annotate(
#     {id: article for id, article in list(eval_set.items())[few_shot_samples:few_shot_samples+test_samples]}
# )

In [12]:
ners = ner.annotate(
    {id: article for id, article in list(eval_set.items())[few_shot_samples:few_shot_samples+test_samples]}
)

Annotating articles:   0%|          | 0/10 [00:00<?, ?it/s]

llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  3343 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   755 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   13562.32 ms /  4098 tokens
Annotating articles:  10%|█         | 1/10 [00:13<02:03, 13.68s/it, id=30099552]Llama.generate: 521 prefix-match hit, remaining 6229 prompt tokens to eval
llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  6229 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1023 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19657.48 ms /  7252 tokens
Annotating articles:  20%|██        | 2/10 [00:33<02:17, 17.21s/it, id=3712

Error in article 37124355
[{'text': 'While parsing a string, we missed the closing quote, ignoring', 'context': ' "abstract'}]
removing last element


llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  3681 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   571 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11803.60 ms /  4252 tokens
Annotating articles:  30%|███       | 3/10 [00:45<01:43, 14.75s/it, id=29870894]Llama.generate: 521 prefix-match hit, remaining 3568 prompt tokens to eval
llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  3568 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   772 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   14425.44 ms /  4340 tokens
Annotating articles:  40%|████      | 4/10 [00:59<01:27, 14.63s/it, id=3822

Error in article 34588398
[{'text': 'While parsing a string, we missed the closing quote, ignoring', 'context': 'ic-induced'}]
removing last element


llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  5341 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1023 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19738.71 ms /  6364 tokens
Annotating articles:  60%|██████    | 6/10 [01:38<01:10, 17.54s/it, id=38432179]Llama.generate: 521 prefix-match hit, remaining 5038 prompt tokens to eval


Error in article 38432179
[{'text': 'While parsing a string, we missed the closing quote, ignoring', 'context': 'cation": "'}]
removing last element


llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  5038 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1023 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19747.02 ms /  6061 tokens
Annotating articles:  70%|███████   | 7/10 [01:58<00:54, 18.27s/it, id=28027925]Llama.generate: 521 prefix-match hit, remaining 5840 prompt tokens to eval


Error in article 28027925
[{'text': 'While parsing a string, we missed the closing quote, ignoring', 'context': ' {\n      "'}]
removing last element


llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  5840 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   828 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   15857.71 ms /  6668 tokens
Annotating articles:  80%|████████  | 8/10 [02:14<00:35, 17.51s/it, id=29315562]Llama.generate: 521 prefix-match hit, remaining 3851 prompt tokens to eval
llama_perf_context_print:        load time =     372.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  3851 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   618 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12066.77 ms /  4469 tokens
Annotating articles:  90%|█████████ | 9/10 [02:26<00:15, 15.81s/it, id=3578

In [13]:
ners

{'30099552': [Entity(start_idx=0, end_idx=14, location='title', text_span='Microorganisms', label='microbiome'),
  Entity(start_idx=-1, end_idx=9, location='title', text_span='Psychiatry', label='DDF'),
  Entity(start_idx=161, end_idx=171, location='abstract', text_span='microbiota', label='microbiome'),
  Entity(start_idx=229, end_idx=239, location='abstract', text_span='microbiome', label='microbiome'),
  Entity(start_idx=944, end_idx=967, location='abstract', text_span='psychological processes', label='DDF'),
  Entity(start_idx=972, end_idx=998, location='abstract', text_span='neuropsychiatric disorders', label='DDF'),
  Entity(start_idx=1014, end_idx=1040, location='abstract', text_span='mood and anxiety disorders', label='DDF'),
  Entity(start_idx=1042, end_idx=1070, location='abstract', text_span='neurodevelopmental disorders', label='DDF'),
  Entity(start_idx=1079, end_idx=1103, location='abstract', text_span='autism spectrum disorder', label='DDF'),
  Entity(start_idx=1108, end

In [ ]:
output_erl_model={
    id: convert_to_output(article).model_dump()
    for id, article in list(annotations.items())
}

In [ ]:
output_erl_model

{'28767318': {'metadata': None,
  'entities': [],
  'relations': None,
  'binary_tag_based_relations': [{'subject_label': 'microbiome',
    'object_label': 'DDF'},
   {'subject_label': 'microbiome', 'object_label': 'DDF'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 

In [ ]:
eval_set

{'30099552': Metadata(title='Making Sense of … the Microbiome in Psychiatry.', author='Thomaz F S Bastiaanssen; Caitlin S M Cowan; Marcus J Claesson; Timothy G Dinan; John F Cryan', journal='The international journal of neuropsychopharmacology', year='2019', abstract="Microorganisms can be found almost anywhere, including in and on the human body. The collection of microorganisms associated with a certain location is called a microbiota, with its collective genetic material referred to as the microbiome. The largest population of microorganisms on the human body resides in the gastrointestinal tract; thus, it is not surprising that the most investigated human microbiome is the human gut microbiome. On average, the gut hosts microbes from more than 60 genera and contains more cells than the human body. The human gut microbiome has been shown to influence many aspects of host health, including more recently the brain.Several modes of interaction between the gut and the brain have been di

In [ ]:
def f1k_annotations(y_true: list[str], y_pred: list[str], k: int = None):
    rel_set = set(y_true)
    # print(rel_set)
    doc_set = set(y_pred[:k])
    tp = len(doc_set.intersection(rel_set))  # docs that are in both -relevant docs
    fp = len(
        doc_set.difference(rel_set)
    )  # docs that are not in relevant set - irrelevant docs (false positiv)
    fn = len(
        rel_set.difference(doc_set)
    )  # relevant docs that are not present in doc set - missing docs
    if tp == 0:
        return 0, 0, 0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return 2 * precision * recall / (precision + recall), precision, recall


f1k_annotations_scores: list[float] = []

for id, article in annotations.items():
    gt_article = article_to_enum_model(eval_set[id], ExtendedEnumERLModel)
    gt_article = convert_to_string_model(gt_article, ExtendedStringERLModel)
    # print(article.relations[0].model_dump_json())
    y_true = [str(r.link_type) for r in gt_article.relations]
    y_pred = [str(r.link_type) for r in article.relations]
    print(y_true)
    print(y_pred)
    scores = f1k_annotations(y_true, y_pred)
    print(scores)
    f1k_annotations_scores.append(scores[0])

np.mean(f1k_annotations_scores)

['LinkType.DDF_strike_anatomical_location', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.DDF_affect_DDF', 'LinkType.DDF_affect_DDF', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.DDF_strike_anatomical_location', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.DDF_affect_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF']
['LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.drug_change_effect_DDF', 'LinkType.drug_change_effect_DDF']
(0.3333333333333333, 0.5, 0.25)
['LinkType.chemical_influence

np.float64(0.2921717171717172)